In [1]:
import os
import sys

from dotenv import load_dotenv
load_dotenv()

# os.environ['HF_HOME']="/Users/nikhil20.sharma/Desktop/langchain/.cache"

# Print all environment variables loaded from .env
print("Loaded Environment Variables:")
for key, value in os.environ.items():
    if key in ['OPENAI_API_KEY', 'LANGSMITH_AIP_KEY', 'HUGGINGFACE_TOKEN']:
        # Mask sensitive values for security
        masked_value = value[:8] + "..." + value[-4:] if value else value
        print(f"- {key}: {masked_value}")

Loaded Environment Variables:
- OPENAI_API_KEY: sk-proj-...znMA
- HUGGINGFACE_TOKEN: hf_Xbepg...LTAM
- LANGSMITH_AIP_KEY: lsv2_pt_...4996


In [ ]:
# QUESTIONS

# Diff b/w PromptTemplate and ChatPromptTemplate and ChatMessagePromptTemplate

## Ways to get structured output - with_structured_output

There are models which are capable of giving structured output and some are not

#### 3 Data formats:
1. TypedDict: Only for representation purpose and for data validation
2. Pydantic: For data validation
3. Json

#### 2 modes can be used while using "with_structured_output"
1. Json Mode (Claud/Gemini)
2. Function Calling Mode (Agents/OpenAI)

### 1. Typed Dict

In [63]:
from typing import TypedDict

class Person(TypedDict):
    name: str
    age: int
    salary: float

new_person: Person = Person(
    name='Nikhil',
    age=20,
    salary=50000
)

new_person

{'name': 'Nikhil', 'age': 20, 'salary': 50000}

In [88]:
from langchain_openai import ChatOpenAI
from typing import TypedDict, List, Optional, Annotated, Literal

In [ ]:
model = ChatOpenAI()
# model = ChatOpenAI(model="gpt-4o-mini", max_tokens=None)#, model_kwargs={'temprature':0.5})

In [ ]:
# Output Schema
class ReviewExtractor(TypedDict):
    summary: Annotated[str, "A brief summary of the review"]
    # sentiment: Annotated[str, "Return setinment of the review, either Positive, Neutral or Negative"]
    sentiment: Annotated[Literal["Positive", 'Negative', 'Neutral'], "Return setinment of the review, either Positive, Neutral or Negative"]
    pros: Annotated[List[Optional[str]], "Write all the pros inside a list"]
    cons: Annotated[List[Optional[str]], "Write all the cons inside a list"]
    # score: float  # Rating between 0 and 10
    score: Annotated[float, "Give rating based on the review. the rating must lie between 0 and 10"]
    themes: Annotated[List[Optional[str]], "Write all the key themes mentioned in the review, in the list format"]

In [83]:
structured_model = model.with_structured_output(ReviewExtractor)

/opt/miniconda3/envs/lang/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [85]:
test_summary1 = """After studying a lot, I finally purchased the S24+. Here is my review after using it for more than one month. I haven't played any games yet, but I have tested the camera in various conditions.
The processor and battery were the main highlights.

No heating issues even in 8K and 4K video recording. As a content creator I have to record long 4k video of more than 1 hour.

Excellent battery backup. Only a 2% battery drain overnight.

The camera is awesome. Photos, portrait video, and ultra-stable videos are next-level. Ultra-stable mode is basically unnecessary as normal video is also very stable.
Primary 50 MP camera shoot slightly better details than iPhone 15. But in 3x and 10x it far better from iPhone 15 because of it's dedicated telephoto lens. Pathin video section iPhone is lightly capture better dynamic range. In in stability both are extraordinary.

The display is ultra smooth and excellent.

Processor:
The Exynos 2400 is definitely better than the Snapdragon 8 Gen 2. Thanks to Samsung for finally providing this improvement in the processor. Maybe the AMD GPU also made a difference.

Finally, I am very happy with this product."""

test_summary2 = """I recently upgraded to the Samsung Galaxy S23 Ultra, and it's been a fantastic experience. The phone is incredibly fast, thanks to its top-tier processor and ample RAM. Multitasking is seamless, and even the most demanding apps run without a hitch.

The display is stunning, with vibrant colors and sharp detail, making it perfect for watching videos or playing games. The 120Hz refresh rate makes everything feel smooth and responsive.

The camera is where this phone truly shines. The 200MP main camera captures exceptional detail, even in low light, and the zoom capabilities are unreal. I can zoom in from far distances and still get clear, usable shots. The S Pen is another standout feature, making it easier to take notes, navigate, and even edit photos.

The battery life is solid, lasting a full day with moderate to heavy use, and fast charging is a lifesaver when I need a quick top-up.

Overall, the S23 Ultra is an all-around powerhouse that excels in performance, display quality, and photography. It’s definitely worth the investment if you're looking for a premium smartphone. Highly recommended!"""

In [86]:
result1 = structured_model.invoke(test_summary1)
result1

{'summary': 'Positive review of the Samsung S24+ after one month of use',
 'sentiment': 'Positive',
 'pros': ['Excellent battery backup',
  'Awesome camera performance in various conditions',
  'Ultra-smooth and excellent display',
  'Improved Exynos 2400 processor compared to Snapdragon 8 Gen 2',
  'Dedicated telephoto lens for better zoom performance'],
 'cons': [],
 'score': 9.5,
 'themes': ['Camera Performance',
  'Battery Life',
  'Display Quality',
  'Processor Performance']}

In [ ]:
result2 = structured_model.invoke(test_summary2)
result2

{'summary': 'An all-around powerhouse with top-tier performance, stunning display, and exceptional camera',
 'sentiment': 'Positive',
 'pros': ['Fast processor and ample RAM for seamless multitasking',
  'Stunning display with vibrant colors and sharp detail',
  '200MP main camera with exceptional detail and unreal zoom capabilities',
  'S Pen for easier note-taking and editing photos',
  'Solid battery life with fast charging'],
 'cons': [],
 'score': 9.5,
 'themes': ['Performance',
  'Display Quality',
  'Camera',
  'Battery Life',
  'S Pen']}

### 2. Pydandic

In [ ]:
from pydantic import BaseModel, EmailStr, Field
from typing import Optional

class Student(BaseModel):
    name: str = "Nikhil Sharma"
    age: Optional[int] = None
    email: EmailStr
    cgpa: float = Field(gt=0, lt=10, default=0, description='CGPA b/w 0 to 10')

student1 = {'name': 'Nikhil', 'age':20, 'email':'asdf@asdf.com', 'cgpa':3.3}
student1 = Student(**student1)
print(student1)

student4 = {'name': 'Nikhil', 'age':'20', 'email':'asdf@asdf.com', 'cgpa':3.3}
student4 = Student(**student4)
print(student4)

student3 = {}
student3 = Student(**student3)
print(student3.name, student3.age)

student2 = {'name': 23}
student2 = Student(**student2)
print(student2)

student1.model_dump()  # Dict object
student1.model_dump_json()


name='Nikhil' age=20 email='asdf@asdf.com' cgpa=3.3
name='Nikhil' age=20 email='asdf@asdf.com' cgpa=3.3


ValidationError: 1 validation error for Student
email
  Field required [type=missing, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [103]:
from langchain_openai import ChatOpenAI
from typing import TypedDict, List, Optional, Annotated, Literal
from pydantic import BaseModel, Field

In [104]:
model = ChatOpenAI()
# model = ChatOpenAI(model="gpt-4o-mini", max_tokens=None)#, model_kwargs={'temprature':0.5})

In [116]:
# Output Schema
class ReviewExtractor(TypedDict):
    themes: list[str] = Field(description="Write all the key themes mentioned in the review, in the list format")
    summary: str = Field(description="A brief summary of the review")
    sentiment: Literal["Positive", "Negative"] = Field(description="Return setinment of the review, either Positive, Neutral or Negative")
    pros: Optional[list[str]] = Field(default=None, description="Write all the pros inside a list")
    cons: Optional[list[str]] = Field(default=None, description="Write all the cons inside a list")
    score: float = Field(default=5, gt=0, lt=10, description="Give rating based on the review. the rating must lie between 0 and 10")
    name: Optional[str] = Field(default=None, description='Write the name of the reviewer if found in the review')

In [117]:
structured_model = model.with_structured_output(ReviewExtractor)

/opt/miniconda3/envs/lang/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [118]:
test_summary1 = """After studying a lot, I finally purchased the S24+. Here is my review after using it for more than one month. I haven't played any games yet, but I have tested the camera in various conditions.
The processor and battery were the main highlights.

No heating issues even in 8K and 4K video recording. As a content creator I have to record long 4k video of more than 1 hour.

Excellent battery backup. Only a 2% battery drain overnight.

The camera is awesome. Photos, portrait video, and ultra-stable videos are next-level. Ultra-stable mode is basically unnecessary as normal video is also very stable.
Primary 50 MP camera shoot slightly better details than iPhone 15. But in 3x and 10x it far better from iPhone 15 because of it's dedicated telephoto lens. Pathin video section iPhone is lightly capture better dynamic range. In in stability both are extraordinary.

The display is ultra smooth and excellent.

Processor:
The Exynos 2400 is definitely better than the Snapdragon 8 Gen 2. Thanks to Samsung for finally providing this improvement in the processor. Maybe the AMD GPU also made a difference.

Finally, I am very happy with this product. Reviewed by Nikhil Sharma"""

test_summary2 = """This is my review Niks, I recently upgraded to the Samsung Galaxy S23 Ultra, and it's been a fantastic experience. The phone is incredibly fast, thanks to its top-tier processor and ample RAM. Multitasking is seamless, and even the most demanding apps run without a hitch.

The display is stunning, with vibrant colors and sharp detail, making it perfect for watching videos or playing games. The 120Hz refresh rate makes everything feel smooth and responsive.

The camera is where this phone truly shines. The 200MP main camera captures exceptional detail, even in low light, and the zoom capabilities are unreal. I can zoom in from far distances and still get clear, usable shots. The S Pen is another standout feature, making it easier to take notes, navigate, and even edit photos.

The battery life is solid, lasting a full day with moderate to heavy use, and fast charging is a lifesaver when I need a quick top-up.

Overall, the S23 Ultra is an all-around powerhouse that excels in performance, display quality, and photography. It’s definitely worth the investment if you're looking for a premium smartphone. Highly recommended!"""

In [113]:
result1 = structured_model.invoke(test_summary1)
result1

{'themes': ['processor', 'battery', 'camera', 'display'],
 'summary': 'The S24+ is a great purchase with highlights in the processor and battery performance. The camera excels in various conditions, offering excellent details in photos and videos. The ultra-stable mode is impressive, and the display is ultra smooth and excellent. Overall, a very happy user.',
 'sentiment': 'Positive',
 'pros': ['Processor performance',
  'Battery backup',
  'Camera quality',
  'Display smoothness'],
 'cons': [],
 'score': 9.5,
 'name': 'Nikhil Sharma'}

In [114]:
result1.name

AttributeError: 'dict' object has no attribute 'name'

In [119]:
result2 = structured_model.invoke(test_summary2)
result2

{'themes': ['performance',
  'display quality',
  'camera',
  'battery life',
  'value for money'],
 'summary': 'The Samsung Galaxy S23 Ultra is a powerhouse smartphone with top-tier performance, stunning display quality, exceptional camera capabilities, solid battery life, and great value for money. Highly recommended!',
 'sentiment': 'Positive',
 'pros': ['Incredibly fast performance',
  'Stunning display with vibrant colors and sharp detail',
  'Exceptional 200MP main camera with unreal zoom capabilities',
  'S Pen for easy note-taking and editing photos',
  'Solid battery life lasting a full day with fast charging',
  'Great value for a premium smartphone'],
 'cons': [],
 'score': 5,
 'name': 'Niks'}

### 3. Json

In [ ]:
# {
#     "title": "student",
#     "description": "Information about student",
#     "type": "object",
#     "properties": {
#         "name": "string",
#         "age": "integer"
#     },
#     "required": ["name"]
# }

In [120]:
from langchain_openai import ChatOpenAI

In [121]:
model = ChatOpenAI()
# model = ChatOpenAI(model="gpt-4o-mini", max_tokens=None)#, model_kwargs={'temprature':0.5})

In [122]:
# schema
json_schema = {
  "title": "Review",
  "type": "object",
  "properties": {
    "key_themes": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Write down all the key themes discussed in the review in a list"
    },
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "enum": ["pos", "neg"],
      "description": "Return sentiment of the review either negative, positive or neutral"
    },
    "pros": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the pros inside a list"
    },
    "cons": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the cons inside a list"
    },
    "name": {
      "type": ["string", "null"],
      "description": "Write the name of the reviewer"
    }
  },
  "required": ["key_themes", "summary", "sentiment"]
}

In [123]:
structured_model = model.with_structured_output(json_schema)

/opt/miniconda3/envs/lang/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1390: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [124]:
result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Niks
""")

In [126]:
type(result)

dict

In [125]:
result

{'key_themes': ['Snapdragon 8 Gen 3 processor',
  'battery life',
  'S-Pen integration',
  '200MP camera',
  'size and weight',
  'Samsung One UI',
  'price'],
 'summary': 'Niks is impressed with the Samsung Galaxy S24 Ultra, highlighting the powerful processor, long battery life, stunning 200MP camera, and useful S-Pen support. However, they find the size and weight uncomfortable for one-handed use, criticize the bloatware in Samsung One UI, and mention the high price tag as a drawback.',
 'sentiment': 'pos',
 'pros': ['Insanely powerful processor (great for gaming and productivity)',
  'Stunning 200MP camera with incredible zoom capabilities',
  'Long battery life with fast charging',
  'S-Pen support is unique and useful'],
 'cons': ['Size and weight make it uncomfortable for one-handed use',
  'Bloatware in Samsung One UI',
  'High price tag'],
 'name': 'Niks'}

## Messages Place holder

In [48]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [53]:
# Chat Template
chat_template = ChatPromptTemplate([
    ('system', 'Your are a customer support agent.'),
    MessagesPlaceholder(variable_name='chat_history'),
    ('user', '{query}')
])
chat_template

ChatPromptTemplate(input_variables=['chat_history', 'query'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[l

In [60]:
# Load Chat History
chat_history = []

with open(file='chat_history.txt', mode='r') as f:
    content = f.readlines()
    content = [x.strip() for x in content]

chat_history.extend(content)
chat_history


["SystemMessage(content='You are a helpful assistant who gives answers in as less words as possible.', additional_kwargs={}, response_metadata={})",
 "HumanMessage(content='Hi', additional_kwargs={}, response_metadata={})",
 "AIMessage(content='Hello! How can I help you?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 27, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-4cf6376e-1d7c-4cca-a807-df6f54b3ae46-0', usage_metadata={'input_tokens': 27, 'output_tokens': 9, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})",
 "HumanMessage(content='

In [61]:
# Create Prompt
prompt = chat_template.invoke({
    'chat_history': chat_history,
    'query': 'What were we discussing?'
})
prompt

ChatPromptValue(messages=[SystemMessage(content='Your are a customer support agent.', additional_kwargs={}, response_metadata={}), HumanMessage(content="SystemMessage(content='You are a helpful assistant who gives answers in as less words as possible.', additional_kwargs={}, response_metadata={})", additional_kwargs={}, response_metadata={}), HumanMessage(content="HumanMessage(content='Hi', additional_kwargs={}, response_metadata={})", additional_kwargs={}, response_metadata={}), HumanMessage(content="AIMessage(content='Hello! How can I help you?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 27, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop'

## Chat Prompt Messages - List of messages - Dynamic messages

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

In [ ]:
# DOES NOT WORK AS INTENDED
chat_template = ChatPromptTemplate([
    SystemMessage(content="You are a helpful chatbot with years of experience in {domain} domain."),
    HumanMessage(content="Explain me in simple terms how {topic} works, as a short poem.")
])

prompt = chat_template.invoke({
    'domain': 'Computer Science',
    'topic': 'Transformers'
})

prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful chatbot with years of experience in {domain} domain.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain me in simple terms how {topic} works, as a short poem.', additional_kwargs={}, response_metadata={})])

In [ ]:
# RECOMMENDED

chat_template = ChatPromptTemplate([
    ('system', 'You are a helpful chatbot with years of experience in {domain} domain.'),
    ('human', 'Explain me in simple terms how {topic} works, as a short poem.')
])

prompt = chat_template.invoke({
    'domain': 'Computer Science',
    'topic': 'Transformers'
})

prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful chatbot with years of experience in Computer Science domain.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain me in simple terms how Transformers works, as a short poem.', additional_kwargs={}, response_metadata={})])

In [47]:
chat_template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful chatbot with years of experience in {domain} domain.'),
    ('human', 'Explain me in simple terms how {topic} works, as a short poem.')
])

prompt = chat_template.invoke({
    'domain': 'Computer Science',
    'topic': 'Transformers'
})

prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful chatbot with years of experience in Computer Science domain.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain me in simple terms how Transformers works, as a short poem.', additional_kwargs={}, response_metadata={})])

## Chat History

In [28]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini", max_tokens=None)  #, model_kwargs={'temprature':0.5})

In [37]:
chat_history = [
    SystemMessage("You are a helpful assistant who gives answers in as less words as possible.")
]

In [38]:
while True:
    user_input = input("You: ")
    if user_input=='exit':
        break
    chat_history.append(HumanMessage(content=user_input))
    result = model.invoke(chat_history)
    print("Bot:", result.content)
    chat_history.append(result)
    

Bot: Hello! How can I help you?
Bot: October 4, 2023.
Bot: Wednesday.
Bot: October.
Bot: I’m called Assistant.
Bot: I don't have an age.
Bot: I'm based on OpenAI's GPT-3.


In [39]:
chat_history

[SystemMessage(content='You are a helpful assistant who gives answers in as less words as possible.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I help you?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 27, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-4cf6376e-1d7c-4cca-a807-df6f54b3ae46-0', usage_metadata={'input_tokens': 27, 'output_tokens': 9, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 HumanMessage(content='What is

## LLM Prompts

In [6]:
import random

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate, load_prompt

In [19]:
model = ChatOpenAI()

In [10]:
paper_input = ["Attention Is All You Need", "BERT: Pre-training of Deep Bidirectional Transformers", "GPT-3: Language Models are Few-Shot Learners", "Diffusion Models Beat GANs on Image Synthesis"]
style_input = ["Beginner-Friendly", "Technical", "Code-Oriented", "Mathematical"]
length_input = ["Too Short (2-3 lines)", "Short (1-2 paragraphs)", "Medium (3-5 paragraphs)", "Long (detailed explanation)"]


In [12]:
paper_input = random.choice(paper_input)
style_input = random.choice(style_input)
length_input = random.choice(length_input)

In [13]:
paper_input, style_input, length_input

('GPT-3: Language Models are Few-Shot Learners',
 'Beginner-Friendly',
 'Too Short (2-3 lines)')

In [ ]:
## METHOD 1

# Design the template

template = PromptTemplate(
    template="""
        Please summarize the research paper titled "{paper_input}" with the following specifications:
        Explanation Style: {style_input}
        Explanation Length: {length_input}
        1. Mathematical Details:  
            - Include relevant mathematical equations if present in the paper.
            - Explain the mathematical concepts using simple, intuitive code snippets where applicable.
        2. Analogies:
            - Use relatable analogies to simplify complex ideas.
        If certain information is not available in the paper, respond with: "Insufficient information available" instead of guessing.
        Ensure the summary is clear, accurate, and aligned with the provided style and length.",
        """, 
    input_variables=[
        'paper_input', 
        'style_input', 
        'length_input'
    ],
    validate_template=True
)

# Render the template
prompt = template.invoke({
    'paper_input': paper_input,
    'style_input': style_input,
    'length_input': length_input
})


result = model.invoke(prompt)

print(result.content)

The research paper discusses GPT-3, a language model that can learn new tasks quickly with limited examples. It uses few-shot learning, allowing it to generalize from a small amount of training data. This is similar to how a student can quickly grasp a new concept with just a few examples.


In [25]:

## METHOD 2

template = load_prompt("prompt_template.json")
chain = template | model
result = chain.invoke({
    'paper_input': paper_input,
    'style_input': style_input,
    'length_input': length_input
})

print(result.content)

The research paper "GPT-3: Language Models are Few-Shot Learners" discusses how GPT-3, a large language model, can learn from just a few examples to perform various tasks effectively. It showcases the impressive few-shot learning capabilities of GPT-3, highlighting its potential for real-world applications in natural language processing. An analogy to understand this would be like a student who can learn and excel in different subjects with minimal study material.


## Prompt Generator

In [1]:
from langchain_core.prompts import PromptTemplate

# Design the template
template = PromptTemplate(
    template="""
        Please summarize the research paper titled "{paper_input}" with the following specifications:
        Explanation Style: {style_input}
        Explanation Length: {length_input}
        1. Mathematical Details:  
            - Include relevant mathematical equations if present in the paper.
            - Explain the mathematical concepts using simple, intuitive code snippets where applicable.
        2. Analogies:
            - Use relatable analogies to simplify complex ideas.
        If certain information is not available in the paper, respond with: "Insufficient information available" instead of guessing.
        Ensure the summary is clear, accurate, and aligned with the provided style and length.",
        """, 
    input_variables=[
        'paper_input', 
        'style_input', 
        'length_input'
    ],
    validate_template=True
)

template.save("prompt_template.json")

## HuggingFace - Text Embeddings - Local

In [40]:
from langchain_huggingface import HuggingFaceEmbeddings

In [42]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [48]:
documents = [
    "Space Exploration: The shimmering expanse of the cosmos beckons, urging humanity to push beyond Earth's bounds, seeking new worlds and unraveling the universe's ancient mysteries.",
    "Artificial Intelligence: As algorithms evolve, the ethical implications of AI's growing autonomy demand careful consideration, ensuring these powerful tools serve humanity's best interests.",
    "Climate Change: Rising sea levels and extreme weather patterns serve as stark reminders that immediate, global action is essential to mitigate the devastating effects of climate change.",
    "Renewable Energy: Harnessing the sun, wind, and geothermal forces offers a sustainable path toward a cleaner future, reducing our reliance on fossil fuels and environmental damage.",
    "Education Reform: Modernizing educational systems to foster critical thinking and creativity empowers future generations to navigate complex challenges in an ever-changing world.",
    "Mental Health Awareness: Breaking down societal stigmas surrounding mental health and providing accessible support systems are crucial for fostering well-being within communities.",
    "Global Connectivity: The internet's vast network connects individuals across continents, enabling instantaneous communication and the rapid exchange of knowledge and cultural ideas.",
    "Urban Development: Sustainable urban planning, incorporating green spaces and efficient infrastructure, is vital for creating livable cities that prioritize residents' quality of life.",
    "Food Security: Addressing global food security requires innovative agricultural practices and equitable distribution systems to ensure everyone has access to nutritious meals.",
    "Artistic Expression: Through diverse mediums, artists challenge perspectives, evoke emotions, and provide unique insights into the human condition, enriching our cultural landscape."
]

In [44]:
query = "Delhi is the capital of India"

In [49]:
documents_embeddings = embedding_model.embed_documents(documents)

In [51]:
len(documents_embeddings), len(documents_embeddings[0])

(10, 384)

In [ ]:
query_embedding = embedding_model.embed_query(text=query)

In [52]:
len(query_embedding), query_embedding[:5]

(384,
 [0.04354957118630409,
  0.02387721836566925,
  -0.045241307467222214,
  0.035404983907938004,
  -0.01665104180574417])

In [53]:
from sklearn.metrics.pairwise import cosine_similarity

In [54]:
scores = cosine_similarity([query_embedding], documents_embeddings)[0]

In [55]:
index, score = sorted(list(enumerate(scores)), key=lambda x: x[1])[-1]

In [57]:
display(f"""
QUERY: {query}
ANS | {round(score, 2)}: {documents[index]}
""")

"\nQUERY: Delhi is the capital of India\nANS | 0.1: Urban Development: Sustainable urban planning, incorporating green spaces and efficient infrastructure, is vital for creating livable cities that prioritize residents' quality of life.\n"

## OpenAI - Text Embeddings

In [26]:
from langchain_openai import OpenAIEmbeddings

In [60]:
embedding_model = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=32)

In [28]:
documents = [
    "Space Exploration: The shimmering expanse of the cosmos beckons, urging humanity to push beyond Earth's bounds, seeking new worlds and unraveling the universe's ancient mysteries.",
    "Artificial Intelligence: As algorithms evolve, the ethical implications of AI's growing autonomy demand careful consideration, ensuring these powerful tools serve humanity's best interests.",
    "Climate Change: Rising sea levels and extreme weather patterns serve as stark reminders that immediate, global action is essential to mitigate the devastating effects of climate change.",
    "Renewable Energy: Harnessing the sun, wind, and geothermal forces offers a sustainable path toward a cleaner future, reducing our reliance on fossil fuels and environmental damage.",
    "Education Reform: Modernizing educational systems to foster critical thinking and creativity empowers future generations to navigate complex challenges in an ever-changing world.",
    "Mental Health Awareness: Breaking down societal stigmas surrounding mental health and providing accessible support systems are crucial for fostering well-being within communities.",
    "Global Connectivity: The internet's vast network connects individuals across continents, enabling instantaneous communication and the rapid exchange of knowledge and cultural ideas.",
    "Urban Development: Sustainable urban planning, incorporating green spaces and efficient infrastructure, is vital for creating livable cities that prioritize residents' quality of life.",
    "Food Security: Addressing global food security requires innovative agricultural practices and equitable distribution systems to ensure everyone has access to nutritious meals.",
    "Artistic Expression: Through diverse mediums, artists challenge perspectives, evoke emotions, and provide unique insights into the human condition, enriching our cultural landscape."
]

In [ ]:
documnet_embeddings = embedding_model.embed_documents(documents)

In [58]:
len(documnet_embeddings), len(documnet_embeddings[0])

(10, 32)

In [59]:
query = "Delhi is the capital of India"

In [61]:
query_embedding = embedding_model.embed_query(query)

In [64]:
len(query_embedding)

32

In [65]:
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
index, score = sorted(list(enumerate(scores)), key=lambda x: x[1])[-1]

In [67]:
display(f"""
QUERY: {query}
ANS | {round(score, 2)}: {documents[index]}
""")

"\nQUERY: Delhi is the capital of India\nANS | 0.1: Urban Development: Sustainable urban planning, incorporating green spaces and efficient infrastructure, is vital for creating livable cities that prioritize residents' quality of life.\n"

## HuggingFace - Local

In [11]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

In [14]:
llm = HuggingFacePipeline.from_model_id(
    model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    task="text-generation",
    pipeline_kwargs=dict(
        temperature=0.9,
        max_new_tokens=100
    )
)

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.hf.co/repos/2b/64/2b642798915fc368e7b638986f68446b121c1d59b30075e146bd6312ee664ac2/6e6001da2106d4757498752a021df6c2bdc332c650aae4bae6b0c004dcf14933?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1741084954&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTA4NDk1NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzJiLzY0LzJiNjQyNzk4OTE1ZmMzNjhlN2I2Mzg5ODZmNjg0NDZiMTIxYzFkNTliMzAwNzVlMTQ2YmQ2MzEyZWU2NjRhYzIvNmU2MDAxZGEyMTA2ZDQ3NTc0OTg3NTJhMDIxZGY2YzJiZGMzMzJjNjUwYWFlNGJhZTZiMGMwMDRkY2YxNDkzMz9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=me7RotsnBwvMQMaydsYL7zwfPg7v85DcMpkLQt8z41b%7EPc7O-6Jg7XoKI4qE4y-8fu1hGm7fHPIqdQqsaZG51b%7EP6sXRmzHSYPq0CR%7ESENqQ8jc5mIaSOLa0SXessY711IWKkAGzh0kkznzBv4Jj5Fcdf%7EdtG2BFFW%7E8NGO%7ET1lLKyQeIr95VfS%7Eszr4oibJszC6R5OQuOsUepQhKyRbEMhqF-KrkXq1WYDkjifUaPT9nfRoo8SxfZ

model.safetensors:  58%|#####7    | 1.27G/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use mps:0


In [15]:
model = ChatHuggingFace(llm=llm)

In [16]:
result = model.invoke("Give me a poem on sun shining in summer, india context. 6 lines only.")

/opt/miniconda3/envs/lang/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/miniconda3/envs/lang/lib/python3.12/site-packages/transformers/pytorch_utils.py:338: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


In [24]:
result.content

"<|user|>\nGive me a poem on sun shining in summer, india context. 6 lines only.</s>\n<|assistant|>\nIn the summer's warmth,\nThe sun beams down upon us,\nA ray of light that shines bright,\nA symbol of hope and joy.\n\nThe sun's rays dance across the sky,\nA symphony of colors,\nA rainbow of hues,\nA sight to behold.\n\nThe warmth of the sun's rays,\nA balm for our weary soul,\nA reminder of the beauty"

In [21]:
print(result.content)

## HuggingFace - Online

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

In [7]:
llm = HuggingFaceEndpoint(
    # repo_id='deepseek-ai/DeepSeek-R1',
    repo_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0',
    task='text-generation'
)

In [8]:
model = ChatHuggingFace(llm=llm)

In [ ]:
result = model.invoke("Give me a poem on sun shining in summer, india context. 6 lines only.")

In [ ]:
print(result.content)

## Open-AI Chat Models

In [23]:
from langchain_openai import ChatOpenAI

In [41]:
model = ChatOpenAI(model="gpt-4o-mini", max_tokens=None)#, model_kwargs={'temprature':0.5})

In [42]:
result = model.invoke(input="Give me a poem on sun shining in summer, india context. 6 lines only.")

In [44]:
print(result.content)

Golden rays dance on fields so wide,  
Mango trees sway with the warm, soft tide.  
Children laugh, their kites soar high,  
Beneath the vast, azure Indian sky.  
Spices mingle in the market's hum,  
In summer's embrace, life's vibrant drum.  


## Open-AI LLM - Not Used Anymore

In [7]:
from langchain_openai import OpenAI, ChatOpenAI

In [12]:
llm = OpenAI(
    model=["babbage-002", "gpt-3.5-turbo-instruct"][1]
    )

In [13]:
result = llm.invoke("What is the Capital of India?")

In [14]:
print(result)



The capital of India is New Delhi.


In [ ]:
# import getpass
# import os

# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [ ]:
# import openai
# from openai import OpenAI
# openai.api_key = os.environ["PERSONAL_OPENAI_API_KEY"]